# Infer-7-Classification : Classification Bayesienne

**Serie** : Programmation Probabiliste avec Infer.NET (7/13)  
**Duree estimee** : 50 minutes  
**Prerequis** : Infer-6-TrueSkill

---

## Objectifs

- Implementer la regression logistique bayesienne
- Comprendre le Bayes Point Machine (BPM)
- Appliquer l'inference bayesienne aux tests cliniques (A/B testing)
- Gerer l'incertitude dans les predictions

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-6-TrueSkill](Infer-6-TrueSkill.ipynb) | [Infer-8-Model-Selection](Infer-8-Model-Selection.ipynb) |

---

## 1. Configuration

Cette section prepare l'environnement pour les modeles de classification bayesienne. Contrairement aux classifieurs deterministes, l'approche bayesienne fournit non seulement une prediction mais aussi une mesure de confiance via des distributions de probabilite.

In [ ]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET pret !");

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET pret !


### Environnement pret

Les packages Infer.NET charges incluent :
- **Microsoft.ML.Probabilistic** : Structures de donnees probabilistes (distributions, variables)
- **Microsoft.ML.Probabilistic.Compiler** : Compilation des modeles en code executable
- **Microsoft.ML.Probabilistic.Math** : Fonctions mathematiques (MMath.NormalCdf pour le probit)

Les algorithmes d'inference disponibles pour la classification :
| Algorithme | Usage | Precision |
|------------|-------|-----------|
| **ExpectationPropagation (EP)** | Modeles probit, BPM | Haute |
| **VariationalMessagePassing (VMP)** | Modeles gaussiens mixtes | Moderee |
| **GibbsSampling** | Modeles complexes | Tres haute (mais lent) |

> **Note** : Pour la classification, EP est generalement prefere car il gere bien les facteurs de troncature (comparaisons avec seuils) inherents aux modeles probit.

## 2. Classification Probabiliste

### Difference avec la classification classique

| Approche | Sortie | Incertitude |
|----------|--------|-------------|
| **Classique** | Classe predite | Non |
| **Probabiliste** | P(classe) | Oui |
| **Bayesienne** | Distribution sur P(classe) | Oui + incertitude sur le modele |

### Avantages bayesiens

- Quantification de l'incertitude
- Regularisation naturelle (priors)
- Mise a jour incrementale
- Pas de surapprentissage si bon prior

## 3. Regression Logistique Bayesienne (1 feature)

### Architecture du modele probit

Le modele de regression logistique bayesienne utilise une variable latente gaussienne :

$$y_i = \mathbb{1}[w \cdot x_i - b + \epsilon_i > 0]$$

ou :
- $w$ est le poids (prior Gaussien)
- $b$ est le seuil (prior Gaussien)
- $\epsilon_i \sim \mathcal{N}(0, 1/\tau)$ est le bruit (precision $\tau$ avec prior Gamma)

**Structure du graphe de facteurs** :

```
poids ~ Gaussian(0, 10)     seuil ~ Gaussian(0, 10)     bruitPrecision ~ Gamma(2, 0.5)
        \                         /                              /
         \                       /                              /
          [score = poids*x - seuil]                            /
                    \                                         /
                     \                                       /
                      [scoreBruite ~ Gaussian(score, bruitPrecision)]
                                      |
                                      v
                              [y = scoreBruite > 0]
```

Cette formulation permet d'utiliser l'Expectation Propagation pour inferer les posterieurs sur `poids` et `seuil`.

In [ ]:
// Donnees : classification binaire avec 1 feature
double[] features = { 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0 };
bool[] labels = { false, false, false, true, false, true, true, true };
int n = features.Length;

// Modele : y = sigmoid(w * x - b)
// Equivalent probit : y = I(w * x - b + noise > 0)

Variable<double> poids = Variable.GaussianFromMeanAndVariance(0, 10).Named("poids");
Variable<double> seuil = Variable.GaussianFromMeanAndVariance(0, 10).Named("seuil");
Variable<double> bruitPrecision = Variable.GammaFromShapeAndScale(2, 0.5).Named("bruit");

Range dataRange = new Range(n);
VariableArray<double> xObs = Variable.Array<double>(dataRange).Named("x");
VariableArray<bool> yObs = Variable.Array<bool>(dataRange).Named("y");

using (Variable.ForEach(dataRange))
{
    Variable<double> score = poids * xObs[dataRange] - seuil;
    Variable<double> scoreBruite = Variable.GaussianFromMeanAndPrecision(score, bruitPrecision);
    yObs[dataRange] = (scoreBruite > 0);
}

xObs.ObservedValue = features;
yObs.ObservedValue = labels;

InferenceEngine moteur = new InferenceEngine(new ExpectationPropagation());
moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Gaussian poidsPost = moteur.Infer<Gaussian>(poids);
Gaussian seuilPost = moteur.Infer<Gaussian>(seuil);

Console.WriteLine("=== Regression Logistique Bayesienne ===");
Console.WriteLine($"\nPoids : {poidsPost}");
Console.WriteLine($"Seuil : {seuilPost}");
Console.WriteLine($"\nInterpretation : classe 1 si feature > {seuilPost.GetMean() / poidsPost.GetMean():F2}");

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
=== Regression Logistique Bayesienne ===

Poids : Gaussian(0,8144, 0,03683)
Seuil : Gaussian(3,378, 0,7493)

Interpretation : classe 1 si feature > 4,15


### Analyse du modèle de régression logistique

**Résultats** :
- Poids ≈ 0.81 (positif → classe 1 augmente avec la feature)
- Seuil ≈ 3.38
- Point de décision : feature > 4.15 → classe 1

**Interprétation géométrique** :
Le modèle probit définit une frontière de décision à `x ≈ 4.15`. Les données montrent effectivement une transition entre les classes autour de x=4-5.

**Incertitude sur les paramètres** :
- σ(poids) ≈ 0.19 → relativement certain
- σ(seuil) ≈ 0.87 → plus incertain

> **Avantage bayésien** : Contrairement à la régression logistique classique qui donne des estimations ponctuelles, nous obtenons des **distributions complètes** sur les paramètres, permettant de propager l'incertitude aux prédictions.

## 4. Prediction avec Incertitude

In [ ]:
// Prediction pour de nouvelles valeurs
double[] nouveauxX = { 2.5, 4.5, 6.5, 9.0 };

Console.WriteLine("=== Predictions avec incertitude ===");
Console.WriteLine();

foreach (double x in nouveauxX)
{
    // Modele de prediction
    Variable<Gaussian> poidsPrior = Variable.Observed(poidsPost);
    Variable<Gaussian> seuilPrior = Variable.Observed(seuilPost);
    
    Variable<double> poidsPred = Variable.Random<double, Gaussian>(poidsPrior);
    Variable<double> seuilPred = Variable.Random<double, Gaussian>(seuilPrior);
    Variable<double> bruitPred = Variable.GammaFromShapeAndScale(2, 0.5);
    
    Variable<double> scorePred = poidsPred * x - seuilPred;
    Variable<double> scoreBruitePred = Variable.GaussianFromMeanAndPrecision(scorePred, bruitPred);
    Variable<bool> predLabel = (scoreBruitePred > 0);
    
    InferenceEngine moteurPred = new InferenceEngine(new ExpectationPropagation());
    moteurPred.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    
    Bernoulli prediction = moteurPred.Infer<Bernoulli>(predLabel);
    Console.WriteLine($"x = {x:F1} : P(classe=1) = {prediction.GetProbTrue():F3}");
}

=== Predictions avec incertitude ===

Compiling model...done.
x = 2,5 : P(classe=1) = 0,218
Compiling model...done.
x = 4,5 : P(classe=1) = 0,561
Compiling model...done.
x = 6,5 : P(classe=1) = 0,822
Compiling model...done.
x = 9,0 : P(classe=1) = 0,951


### Interpretation des predictions probabilistes

| Feature (x) | P(classe=1) | Interpretation |
|-------------|-------------|----------------|
| 2.5 | 0.218 | Tres probablement classe 0 |
| 4.5 | 0.561 | Zone d'incertitude (proche de 0.5) |
| 6.5 | 0.822 | Probablement classe 1 |
| 9.0 | 0.951 | Tres probablement classe 1 |

**Observations cles** :

1. **Gradient de confiance** : La probabilite augmente de maniere monotone avec x, coherent avec le poids positif appris.

2. **Zone de transition** : Autour de x=4.5, le modele est incertain (P proche de 0.5). Cela correspond au point de decision calcule (x=4.15).

3. **Propagation de l'incertitude** : Ces probabilites ne sont pas juste `sigmoid(w*x - b)` avec des parametres fixes. Elles integrent l'incertitude sur w et b apprise pendant l'entrainement.

> **Difference avec la classification deterministe** : Un classifieur classique donnerait une prediction binaire (0 ou 1). Ici, nous obtenons une probabilite calibree qui reflette notre confiance reelle basee sur les donnees disponibles.

## 5. Classification Multi-Features

### Extension aux dimensions superieures

Passer d'une seule feature a plusieurs features generalise le modele :

| Aspect | 1 feature | p features |
|--------|-----------|------------|
| Parametre | $w$ (scalaire) | $\mathbf{w}$ (vecteur) |
| Score | $w \cdot x - b$ | $\mathbf{w}^T \mathbf{x} - b$ |
| Frontiere | Point sur $\mathbb{R}$ | Hyperplan dans $\mathbb{R}^p$ |
| Priors | 2 Gaussiennes | p+1 Gaussiennes |

Le graphe de facteurs s'etend naturellement avec un **produit scalaire** entre le vecteur de poids et le vecteur de features.

In [ ]:
// Donnees avec 2 features
double[,] featuresMulti = {
    { 1.0, 2.0 },
    { 2.0, 1.5 },
    { 1.5, 3.0 },
    { 3.0, 3.5 },
    { 4.0, 2.0 },
    { 3.5, 4.0 },
    { 5.0, 3.0 },
    { 4.5, 5.0 }
};
bool[] labelsMulti = { false, false, false, true, false, true, true, true };

int nSamples = labelsMulti.Length;
int nFeatures = 2;

Range sampleRange = new Range(nSamples).Named("sample");
Range featureRange = new Range(nFeatures).Named("feature");

// Poids pour chaque feature
VariableArray<double> poidsMulti = Variable.Array<double>(featureRange).Named("poids");
poidsMulti[featureRange] = Variable.GaussianFromMeanAndVariance(0, 10).ForEach(featureRange);

Variable<double> seuilMulti = Variable.GaussianFromMeanAndVariance(0, 10).Named("seuil");

VariableArray2D<double> xMulti = Variable.Array<double>(sampleRange, featureRange).Named("x");
VariableArray<bool> yMulti = Variable.Array<bool>(sampleRange).Named("y");

using (Variable.ForEach(sampleRange))
{
    // Score = somme(poids[f] * x[f]) - seuil
    Variable<double> scoreMulti = Variable.Sum(
        Variable.Array<double>(featureRange).Named("produit"));
    
    // Alternative plus simple : calculer explicitement
    Variable<double> score0 = poidsMulti[0] * xMulti[sampleRange, 0];
    Variable<double> score1 = poidsMulti[1] * xMulti[sampleRange, 1];
    Variable<double> scoreTot = score0 + score1 - seuilMulti;
    Variable<double> scoreNoise = Variable.GaussianFromMeanAndVariance(scoreTot, 1);
    yMulti[sampleRange] = (scoreNoise > 0);
}

xMulti.ObservedValue = featuresMulti;
yMulti.ObservedValue = labelsMulti;

InferenceEngine moteurMulti = new InferenceEngine(new ExpectationPropagation());
moteurMulti.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Gaussian[] poidsPostMulti = moteurMulti.Infer<Gaussian[]>(poidsMulti);
Gaussian seuilPostMulti = moteurMulti.Infer<Gaussian>(seuilMulti);

Console.WriteLine("=== Classification Multi-Features ===");
for (int f = 0; f < nFeatures; f++)
{
    Console.WriteLine($"Poids feature {f+1} : {poidsPostMulti[f]}");
}
Console.WriteLine($"Seuil : {seuilPostMulti}");

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
=== Classification Multi-Features ===
Poids feature 1 : Gaussian(0,5454, 0,06499)
Poids feature 2 : Gaussian(1,254, 0,07999)
Seuil : Gaussian(5,058, 0,6716)


### Analyse des poids multi-features

**Resultats obtenus** :
- Poids feature 1 (x1) : 0.55 ± 0.25
- Poids feature 2 (x2) : 1.25 ± 0.28
- Seuil : 5.06 ± 0.82

**Interpretation geometrique** :

La frontiere de decision est definie par l'equation :
$$0.55 \cdot x_1 + 1.25 \cdot x_2 - 5.06 = 0$$

Soit : $x_2 = -0.44 \cdot x_1 + 4.05$

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| Pente frontiere | -0.44 | Frontiere quasi-horizontale |
| Intercept | 4.05 | Coupe l'axe x2 vers 4 |
| Ratio poids | 2.3 | Feature 2 a 2.3x plus d'influence que feature 1 |

> **Importance relative** : Le poids de la feature 2 est plus de deux fois celui de la feature 1, indiquant que x2 est plus discriminant pour la classification. Cela pourrait orienter la collecte de donnees futures.

## 6. Bayes Point Machine (BPM)

### Principe

Le BPM est une methode de classification bayesienne qui :
- Marginalise sur tous les hyperplans separateurs possibles
- Donne des probabilites calibrees
- Utilise EP pour l'inference

### Formulation

$$P(y=1|x) = \int P(y=1|x,w) P(w|D) dw$$

### Note technique : Modele probit vs logit

Le code precedent utilise un **modele probit** (comparaison avec bruit gaussien) plutot qu'un modele logit (fonction sigmoid). Voici la difference :

| Aspect | Probit | Logit |
|--------|--------|-------|
| Lien | $\Phi^{-1}(p) = w^T x$ | $\log\frac{p}{1-p} = w^T x$ |
| Distribution latente | Gaussienne | Logistique |
| Inference bayesienne | Plus facile (EP) | Plus difficile |
| Equivalence pratique | Quasi-identique pour la plupart des cas | |

En Infer.NET, le modele probit est prefere car il se prete naturellement a l'Expectation Propagation avec des distributions gaussiennes.

$$P(y=1|x) = \Phi(w^T x) = \int_{-\infty}^{w^T x} \mathcal{N}(z|0,1) dz$$

ou $\Phi$ est la fonction de repartition de la loi normale standard.

In [ ]:
// Bayes Point Machine simplifie

public class SimpleBPM
{
    private int nFeatures;
    private Gaussian[] poidsPosteriors;
    private Gaussian seuilPosterior;
    private InferenceEngine moteur;
    
    public SimpleBPM(int nFeatures)
    {
        this.nFeatures = nFeatures;
        this.moteur = new InferenceEngine(new ExpectationPropagation());
        this.moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    }
    
    public void Entrainer(double[,] X, bool[] y)
    {
        int n = y.Length;
        Range sampleRange = new Range(n);
        Range featureRange = new Range(nFeatures);
        
        VariableArray<double> poids = Variable.Array<double>(featureRange);
        poids[featureRange] = Variable.GaussianFromMeanAndVariance(0, 1).ForEach(featureRange);
        
        Variable<double> seuil = Variable.GaussianFromMeanAndVariance(0, 1);
        
        VariableArray2D<double> xVar = Variable.Array<double>(sampleRange, featureRange);
        VariableArray<bool> yVar = Variable.Array<bool>(sampleRange);
        
        using (Variable.ForEach(sampleRange))
        {
            Variable<double> score = Variable.Constant(0.0);
            for (int f = 0; f < nFeatures; f++)
            {
                score = score + poids[f] * xVar[sampleRange, f];
            }
            score = score - seuil;
            Variable<double> scoreNoise = Variable.GaussianFromMeanAndVariance(score, 1);
            yVar[sampleRange] = (scoreNoise > 0);
        }
        
        xVar.ObservedValue = X;
        yVar.ObservedValue = y;
        
        poidsPosteriors = moteur.Infer<Gaussian[]>(poids);
        seuilPosterior = moteur.Infer<Gaussian>(seuil);
    }
    
    public double Predire(double[] x)
    {
        // Score moyen
        double scoreMoyen = 0;
        double scoreVariance = 0;
        
        for (int f = 0; f < nFeatures; f++)
        {
            scoreMoyen += poidsPosteriors[f].GetMean() * x[f];
            scoreVariance += poidsPosteriors[f].GetVariance() * x[f] * x[f];
        }
        scoreMoyen -= seuilPosterior.GetMean();
        scoreVariance += seuilPosterior.GetVariance() + 1;  // +1 pour le bruit
        
        // Probit : P(score + noise > 0) - utiliser MMath.NormalCdf
        return MMath.NormalCdf(scoreMoyen / Math.Sqrt(scoreVariance));
    }
}

Console.WriteLine("Classe SimpleBPM definie.");

Classe SimpleBPM definie.


### Implementation du Bayes Point Machine

La classe `SimpleBPM` encapsule le workflow complet :

1. **Entrainement** (`Entrainer`) :
   - Definit les priors Gaussiens sur les poids et le seuil
   - Construit le graphe de facteurs avec le modele probit
   - Execute l'inference EP pour obtenir les posterieurs

2. **Prediction** (`Predire`) :
   - Calcule le score moyen : $\mu_{score} = \mathbf{w}_{post}^T \mathbf{x} - b_{post}$
   - Calcule la variance totale : $\sigma^2_{score} = \sum_i \text{Var}(w_i) x_i^2 + \text{Var}(b) + 1$
   - Retourne $\Phi(\mu_{score} / \sigma_{score})$ via `MMath.NormalCdf`

**Formule de prediction avec incertitude** :

$$P(y=1|\mathbf{x}) = \Phi\left(\frac{\mathbf{\mu}_w^T \mathbf{x} - \mu_b}{\sqrt{\mathbf{x}^T \Sigma_w \mathbf{x} + \sigma_b^2 + 1}}\right)$$

ou $\Phi$ est la CDF de la loi normale standard.

### Validation sur les donnees 2D

Testons maintenant notre classe `SimpleBPM` sur les donnees multi-features definies precedemment pour valider l'implementation. Le BPM devrait donner des predictions coherentes avec l'inference directe, mais avec une API plus simple pour les predictions.

In [ ]:
// Utilisation du BPM
var bpm = new SimpleBPM(2);
bpm.Entrainer(featuresMulti, labelsMulti);

Console.WriteLine("=== Predictions BPM ===");
double[][] testPoints = {
    new[] { 1.0, 1.0 },
    new[] { 3.0, 3.0 },
    new[] { 5.0, 4.0 },
    new[] { 2.0, 4.0 }
};

foreach (var point in testPoints)
{
    double prob = bpm.Predire(point);
    Console.WriteLine($"({point[0]}, {point[1]}) : P(classe=1) = {prob:F3}");
}

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
=== Predictions BPM ===
(1, 1) : P(classe=1) = 0,268
(3, 3) : P(classe=1) = 0,603
(5, 4) : P(classe=1) = 0,751
(2, 4) : P(classe=1) = 0,633


### Analyse des predictions BPM

| Point (x1, x2) | P(classe=1) | Position relative | Verdict |
|----------------|-------------|-------------------|---------|
| (1, 1) | 0.268 | Loin sous la frontiere | Classe 0 |
| (3, 3) | 0.603 | Proche de la frontiere | Incertain |
| (5, 4) | 0.751 | Au-dessus de la frontiere | Classe 1 |
| (2, 4) | 0.633 | Legèrement au-dessus | Classe 1 (incertain) |

**Caracteristiques du Bayes Point Machine** :

1. **Calibration** : Les probabilites proches de 0.5 indiquent correctement les zones d'incertitude pres de la frontiere.

2. **Marginalisation** : Le BPM considere tous les hyperplans plausibles ponderes par leur probabilite, pas un seul hyperplan optimal.

3. **Regularisation implicite** : Le prior Gaussien sur les poids (variance = 1) empeche le surapprentissage en penalisant les poids extremes.

> **Comparaison SVM vs BPM** : Un SVM donnerait une frontiere "dure" et une classification binaire. Le BPM fournit des probabilites calibrees, particulierement utiles quand le cout d'erreur est asymetrique ou quand il faut prioriser les cas incertains pour une revue humaine.

## 7. Test Clinique Bayesien (A/B Testing)

### Contexte

Comparer l'efficacite d'un nouveau traitement vs placebo.

### Approche bayesienne

- Prior sur l'efficacite de chaque traitement
- Mise a jour avec les observations
- Probabilite que le traitement soit meilleur

In [ ]:
// Test clinique A/B bayesien

// Donnees observees
int nPlacebo = 100;
int guerisPlacebo = 30;  // 30% guerison

int nTraitement = 100;
int guerisTraitement = 45;  // 45% guerison

// Modele : taux de guerison pour chaque groupe
Variable<double> tauxPlacebo = Variable.Beta(1, 1).Named("tauxPlacebo");  // Prior uniforme
Variable<double> tauxTraitement = Variable.Beta(1, 1).Named("tauxTraitement");

// Observations (distribution binomiale)
Variable<int> obsPlacebo = Variable.Binomial(nPlacebo, tauxPlacebo);
Variable<int> obsTraitement = Variable.Binomial(nTraitement, tauxTraitement);

obsPlacebo.ObservedValue = guerisPlacebo;
obsTraitement.ObservedValue = guerisTraitement;

// Traitement est meilleur ?
Variable<bool> traitementMeilleur = (tauxTraitement > tauxPlacebo);

InferenceEngine moteurClinique = new InferenceEngine();
moteurClinique.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Beta tauxPlaceboPost = moteurClinique.Infer<Beta>(tauxPlacebo);
Beta tauxTraitementPost = moteurClinique.Infer<Beta>(tauxTraitement);
Bernoulli traitementMeilleurPost = moteurClinique.Infer<Bernoulli>(traitementMeilleur);

Console.WriteLine("=== Test Clinique Bayesien ===");
Console.WriteLine($"\nPlacebo : {guerisPlacebo}/{nPlacebo} guerisons");
Console.WriteLine($"Traitement : {guerisTraitement}/{nTraitement} guerisons");

Console.WriteLine($"\nTaux placebo : {tauxPlaceboPost}");
Console.WriteLine($"  Moyenne : {tauxPlaceboPost.GetMean():F3}");
Console.WriteLine($"  IC 95% : [{tauxPlaceboPost.GetMean() - 2*Math.Sqrt(tauxPlaceboPost.GetVariance()):F3}, {tauxPlaceboPost.GetMean() + 2*Math.Sqrt(tauxPlaceboPost.GetVariance()):F3}]");

Console.WriteLine($"\nTaux traitement : {tauxTraitementPost}");
Console.WriteLine($"  Moyenne : {tauxTraitementPost.GetMean():F3}");

Console.WriteLine($"\nP(traitement meilleur) = {traitementMeilleurPost.GetProbTrue():F3}");

Compiling model...compilation had 1 warning(s).
  [1] DifferenceBetaOp.DifferenceAverageConditional(tauxTraitement_uses_F[1], tauxPlacebo_uses_F[1]) has quality band Experimental which is less than the recommended quality band (Preview)
done.
=== Test Clinique Bayesien ===

Placebo : 30/100 guerisons
Traitement : 45/100 guerisons

Taux placebo : Beta(31,71)[mean=0,3039]
  Moyenne : 0,304
  IC 95% : [0,213, 0,395]

Taux traitement : Beta(46,56)[mean=0,451]
  Moyenne : 0,451

P(traitement meilleur) = 0,986


### Modele conjugue Beta-Binomial

Le test clinique utilise le modele classique Beta-Binomial :

**Prior** : $\theta \sim \text{Beta}(1, 1)$ (uniforme sur [0,1])

**Vraisemblance** : $k | \theta \sim \text{Binomial}(n, \theta)$

**Posterior** : $\theta | k \sim \text{Beta}(1 + k, 1 + n - k)$

Cette conjugaison permet une inference exacte. La question "le traitement est-il meilleur ?" se traduit par :
$$P(\theta_{traitement} > \theta_{placebo} | \text{donnees})$$

Infer.NET calcule cette probabilite via l'operateur `DifferenceBetaOp`.

### Analyse du test clinique

**Résultats** :
- Placebo : 30% guérison (IC 95% : 21%-39%)
- Traitement : 45% guérison (IC 95% : 35%-55%)
- **P(traitement meilleur) = 0.986**

**Comparaison avec l'approche fréquentiste** :

| Aspect | Fréquentiste | Bayésien |
|--------|--------------|----------|
| Question | "Peut-on rejeter H0 ?" | "Quelle est P(traitement meilleur) ?" |
| Réponse | p-value < 0.05 → significatif | P = 98.6% → très probable |
| Interprétation | Difficile à communiquer | Directe et intuitive |

**Note importante** : Le warning "quality band Experimental" indique que l'opérateur `DifferenceBetaOp` est encore en développement dans Infer.NET. Les résultats restent fiables mais cette fonctionnalité pourrait évoluer.

> **Application clinique** : Avec 98.6% de probabilité que le traitement soit meilleur, un comité d'éthique pourrait recommander de proposer le traitement au groupe placebo.

## 8. Effet de la Taille d'Echantillon

### Puissance statistique bayesienne

Une question naturelle est : "Combien de patients faut-il recruter pour detecter une difference ?" L'analyse suivante montre comment la certitude evolue avec la taille de l'echantillon, a effet constant (30% vs 45%).

In [ ]:
// Impact de la taille d'echantillon

Console.WriteLine("=== Impact de la taille d'echantillon ===");
Console.WriteLine("\nMeme ratio (30% vs 45%), differentes tailles :\n");

int[] tailles = { 10, 50, 100, 500, 1000 };

foreach (int n in tailles)
{
    int gP = (int)(n * 0.30);
    int gT = (int)(n * 0.45);
    
    Variable<double> tP = Variable.Beta(1, 1);
    Variable<double> tT = Variable.Beta(1, 1);
    
    Variable.ConstrainEqual(Variable.Binomial(n, tP), gP);
    Variable.ConstrainEqual(Variable.Binomial(n, tT), gT);
    
    Variable<bool> meilleur = (tT > tP);
    
    InferenceEngine m = new InferenceEngine();
    m.Compiler.CompilerChoice = CompilerChoice.Roslyn;
    
    double prob = m.Infer<Bernoulli>(meilleur).GetProbTrue();
    Console.WriteLine($"n = {n,4} : P(traitement meilleur) = {prob:F4}");
}

Console.WriteLine("\n=> Plus de donnees = plus de certitude");

=== Impact de la taille d'echantillon ===

Meme ratio (30% vs 45%), differentes tailles :

Compiling model...compilation had 1 warning(s).
  [1] DifferenceBetaOp.DifferenceAverageConditional(vdouble105_uses_F[1], vdouble104_uses_F[1]) has quality band Experimental which is less than the recommended quality band (Preview)
done.
n =   10 : P(traitement meilleur) = 0,6702
Compiling model...compilation had 1 warning(s).
  [1] DifferenceBetaOp.DifferenceAverageConditional(vdouble108_uses_F[1], vdouble107_uses_F[1]) has quality band Experimental which is less than the recommended quality band (Preview)
done.
n =   50 : P(traitement meilleur) = 0,9258
Compiling model...compilation had 1 warning(s).
  [1] DifferenceBetaOp.DifferenceAverageConditional(vdouble111_uses_F[1], vdouble110_uses_F[1]) has quality band Experimental which is less than the recommended quality band (Preview)
done.
n =  100 : P(traitement meilleur) = 0,9862
Compiling model...compilation had 1 warning(s).
  [1] DifferenceBe

### Convergence de la certitude avec la taille d'echantillon

| Taille (n) | P(traitement meilleur) | Interpretation |
|------------|------------------------|----------------|
| 10 | 0.670 | Faible evidence (proche du hasard) |
| 50 | 0.926 | Evidence moderee |
| 100 | 0.986 | Evidence forte |
| 500 | ~1.000 | Evidence tres forte |
| 1000 | ~1.000 | Evidence quasi-certaine |

**Loi de convergence** :

La certitude croit approximativement comme $\sqrt{n}$ :
- Doubler la confiance necessite quadrupler l'echantillon
- Passer de 67% a 99% necessite ~100x plus de donnees

**Implications pratiques** :

| Contexte | Taille recommandee | Justification |
|----------|-------------------|---------------|
| Test pilote | n = 50 | Detection effet important (>90%) |
| Essai clinique | n = 100-500 | Standard reglementaire |
| Decision critique | n > 500 | Minimiser risque d'erreur |

> **Avantage bayesien** : Contrairement aux tests frequentistes qui donnent une reponse binaire (significatif/non-significatif), l'approche bayesienne permet un **arret adaptatif** : on peut arreter l'essai des que P(traitement meilleur) depasse un seuil predetermine (ex: 95%), ou au contraire continuer si l'evidence est insuffisante.

## 9. Exercice : Classification Spam

### Enonce

Construisez un classificateur bayesien pour detecter les spams bases sur 3 features :
- Nombre de mots en majuscules
- Presence du mot "gratuit"
- Longueur du message (en centaines de caracteres)

### Donnees

### Objectif de l'exercice

Construire un classificateur de spam en utilisant le Bayes Point Machine implemente precedemment. Les features choisies representent des caracteristiques typiques des spams :

| Feature | Description | Valeur typique spam |
|---------|-------------|---------------------|
| Majuscules | Nombre de mots en majuscules | Eleve (>15) |
| Gratuit | Presence du mot "gratuit" (0/1) | 1 |
| Longueur | Taille du message (en centaines de caracteres) | Faible (<2) |

Ces features sont simples mais illustrent les principes d'un filtre anti-spam reel.

In [ ]:
// EXERCICE : Classification spam

// Donnees d'entrainement
// [nbMajuscules, presenceGratuit (0/1), longueur]
double[,] spamFeatures = {
    { 5, 0, 2.5 },   // Non spam
    { 3, 0, 3.0 },   // Non spam
    { 15, 1, 1.0 },  // Spam
    { 20, 1, 0.5 },  // Spam
    { 2, 0, 4.0 },   // Non spam
    { 25, 1, 1.5 },  // Spam
    { 8, 0, 2.0 },   // Non spam
    { 18, 1, 0.8 }   // Spam
};
bool[] spamLabels = { false, false, true, true, false, true, false, true };

// Entrainement
var spamClassifier = new SimpleBPM(3);
spamClassifier.Entrainer(spamFeatures, spamLabels);

Console.WriteLine("=== Classificateur Spam ===");

// Test sur nouveaux emails
var testEmails = new (double[], string)[] {
    (new[] { 4.0, 0.0, 3.0 }, "Email normal"),
    (new[] { 22.0, 1.0, 1.0 }, "OFFRE GRATUITE!!!"),
    (new[] { 10.0, 0.0, 2.5 }, "Email avec quelques majuscules"),
    (new[] { 30.0, 1.0, 0.5 }, "CLIQUEZ ICI GRATUIT")
};

Console.WriteLine("\nPredictions :");
foreach (var (features, desc) in testEmails)
{
    double probSpam = spamClassifier.Predire(features);
    string verdict = probSpam > 0.5 ? "SPAM" : "OK";
    Console.WriteLine($"  {desc,-30} : P(spam)={probSpam:F3} -> {verdict}");
}

Compiling model...done.
Iterating: 
.........|.........|.........|.........|.........| 50
=== Classificateur Spam ===

Predictions :
  Email normal                   : P(spam)=0,053 -> OK
  OFFRE GRATUITE!!!              : P(spam)=0,952 -> SPAM
  Email avec quelques majuscules : P(spam)=0,334 -> OK
  CLIQUEZ ICI GRATUIT            : P(spam)=0,984 -> SPAM


### Analyse du classificateur spam

**Resultats de prediction** :

| Email | Majuscules | Gratuit | Longueur | P(spam) | Verdict |
|-------|------------|---------|----------|---------|---------|
| Email normal | 4 | Non | 3.0 | 0.053 | OK |
| OFFRE GRATUITE!!! | 22 | Oui | 1.0 | 0.952 | SPAM |
| Quelques majuscules | 10 | Non | 2.5 | 0.334 | OK |
| CLIQUEZ ICI GRATUIT | 30 | Oui | 0.5 | 0.984 | SPAM |

**Facteurs discriminants appris** :

Le modele a appris que le spam est caracterise par :
1. **Nombre eleve de majuscules** (poids positif)
2. **Presence du mot "gratuit"** (poids positif fort)
3. **Messages courts** (poids negatif sur la longueur)

**Cas interessants** :

- "Email avec quelques majuscules" (P=0.334) : Malgre 10 majuscules, l'absence de "gratuit" et la longueur normale le sauvent du classement spam.
- Les deux emails avec "gratuit" ont P(spam) > 95%, montrant l'importance de cette feature.

> **Application industrielle** : Ce type de classificateur bayesien est utilise dans les filtres anti-spam reels. L'avantage de l'approche probabiliste est de pouvoir definir un seuil de decision adapte au contexte : seuil bas (0.3) pour un filtre agressif, seuil haut (0.8) pour eviter les faux positifs dans un contexte professionnel.

## 10. Resume et Synthese

### Concepts cles

| Concept | Description |
|---------|-------------|
| **Regression logistique bayesienne** | Priors sur poids, posterieurs apres donnees |
| **Bayes Point Machine** | Marginalisation sur hyperplans |
| **Test A/B bayesien** | P(traitement meilleur) directement |
| **Incertitude** | Quantifiee a chaque etape |
| **Calibration** | Probabilites refletent la vraie incertitude |

### Distributions utilisees

| Distribution | Role | Parametres |
|--------------|------|------------|
| **Gaussian** | Prior/Posterior sur poids | (moyenne, variance) |
| **Gamma** | Prior sur precision du bruit | (shape, scale) |
| **Beta** | Prior/Posterior sur probabilites | (alpha, beta) |
| **Bernoulli** | Prediction de classe | (probTrue) |
| **Binomial** | Comptage de succes | (n, p) |

### Comparaison des approches

| Aspect | Classification classique | Classification bayesienne |
|--------|-------------------------|---------------------------|
| Sortie | Classe predite | Distribution sur les classes |
| Incertitude modele | Non | Oui (distributions sur parametres) |
| Incertitude prediction | Non | Oui (probabilites calibrees) |
| Regularisation | Explicite (L1/L2) | Implicite (priors) |
| Surapprentissage | Risque eleve | Reduit naturellement |
| Interpretabilite | Limitee | Elevee (intervalles de credibilite) |

---

## Prochaine etape

Dans [Infer-8-Model-Selection](Infer-8-Model-Selection.ipynb), nous explorerons :

- La selection et comparaison de modeles
- L'evidence bayesienne (marginal likelihood)
- Le facteur de Bayes
- L'Automatic Relevance Determination (ARD)